In [ ]:
!pip install tensorflow
!pip install keras
!pip install keras-tuner

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Conv1D, MaxPooling1D, Flatten, Dense, InputLayer, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.losses import mean_squared_error # Changed the import path to keras.losses
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
import kerastuner as kt

In [ ]:
#load data
data_main = pd.read_csv('continuous dataset - Copy.csv')
data_main.index = pd.to_datetime(data_main['datetime'], format='%Y-%m-%d %H:%M:%S')
columns_to_keep = ['datetime', 'nat_demand', 'T2M_toc', 'QV2M_toc', 'W2M_toc']
data_main = data_main[columns_to_keep]
data_main

,datetime,nat_demand,T2M_toc,QV2M_toc,W2M_toc
datetime,,,,,
2015-01-03 01:00:00,2015-01-03 01:00:00,970.3450,25.865259,0.018576,21.850546
2015-01-03 02:00:00,2015-01-03 02:00:00,912.1755,25.899255,0.018653,22.166944
2015-01-03 03:00:00,2015-01-03 03:00:00,900.2688,25.937280,0.018768,22.454911
2015-01-03 04:00:00,2015-01-03 04:00:00,889.9538,25.957544,0.018890,22.110481
2015-01-03 05:00:00,2015-01-03 05:00:00,893.6865,25.973840,0.018981,21.186089
...,...,...,...,...,...
2020-06-26 20:00:00,2020-06-26 20:00:00,1128.5592,27.246545,0.020303,9.289304
2020-06-26 21:00:00,2020-06-26 21:00:00,1112.7488,27.099573,0.020395,9.837504
2020-06-26 22:00:00,2020-06-26 22:00:00,1081.5680,26.971155,0.020448,10.262464


In [ ]:
#preprocessing and feature engineering
data = pd.DataFrame({'demand': data_main['nat_demand']})
data['date'] = data_main.index
data['day_of_year'] = data['date'].dt.day_of_year
data['month'] = data['date'].dt.month
data['day_of_month'] = data['date'].dt.day
data['day_of_week'] = data['date'].dt.day_of_week
data['hour_of_day'] = data['date'].dt.hour

#additional parameters
data['temp'] = data_main['T2M_toc']
data['hum'] = data_main['QV2M_toc']
data['wind_vel'] = data_main['W2M_toc']

#holiday feature
holidays = ['2015-12-25', '2016-01-01', '2016-12-25', '2017-01-01', '2018-12-25', '2019-01-01', '2020-12-25', '2020-01-01']
data['holiday'] = data['date'].isin(pd.to_datetime(holidays)).astype(int)

#mark saturdays as holidays
data['holiday'] = np.where(data['day_of_week'] == 5, 1, data['holiday'])  # 5 corresponds to Saturday

In [ ]:
#periodic features
data['seconds'] = data['date'].map(pd.Timestamp.timestamp)
day = 60*60*24
year = day*365.2425
data['day_sin'] = np.sin(data['seconds'] * (2 * np.pi / day))
data['day_cos'] = np.cos(data['seconds'] * (2 * np.pi / day))
data['year_sin'] = np.sin(data['seconds'] * (2 * np.pi / year))
data['year_cos'] = np.cos(data['seconds'] * (2 * np.pi / year))
data = data.drop('seconds', axis=1)



In [ ]:
#prepare the dataset for the model
def to_x_y(data, window_size=24):
    data_numpy = data.to_numpy()
    x = []
    y = []
    for i in range(len(data_numpy) - window_size):
        x.append(data_numpy[i:i + window_size])
        y.append(data_numpy[i + window_size, 0])  # demand is the target
    return np.array(x), np.array(y)

In [ ]:
window_size = 24
x, y = to_x_y(data, window_size)

In [ ]:
#split the data into training, validation, and test sets
train_size = int(len(x) * 0.7)
val_size = int(len(x) * 0.2)
test_size = len(x) - train_size - val_size

x_train, x_val, x_test = x[:train_size], x[train_size:train_size+val_size], x[train_size+val_size:]
y_train, y_val, y_test = y[:train_size], y[train_size:train_size+val_size], y[train_size+val_size:]



In [ ]:
#normalize the data
mean_values = np.mean(x_train, axis=0)
std_values = np.std(x_train, axis=0)

In [ ]:
def preprocess(X):
    return (X - mean_values) / std_values

In [ ]:
x_train = preprocess(x_train)
x_val = preprocess(x_val)
x_test = preprocess(x_test)

In [ ]:
#define hypermodel for LSTM tuning
def build_lstm_model(hp):
    model = Sequential()
    model.add(InputLayer(input_shape=(window_size, x_train.shape[2])))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32), return_sequences=True))
    model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32)))
    model.add(Dense(hp.Int('dense_units', min_value=16, max_value=128, step=16), activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss=mean_squared_error, optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), metrics=[RootMeanSquaredError()])
    return model

#define hypermodel for CNN-LSTM tuning
def build_cnn_lstm_model(hp):
    model = Sequential()
    model.add(InputLayer(input_shape=(window_size, x_train.shape[2])))
    model.add(Conv1D(filters=hp.Int('filters', min_value=32, max_value=128, step=32), kernel_size=hp.Int('kernel_size', min_value=2, max_value=4, step=1), activation='relu'))
    model.add(MaxPooling1D(pool_size=hp.Int('pool_size', min_value=2, max_value=4, step=1)))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32), return_sequences=True))
    model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32)))
    model.add(Dense(hp.Int('dense_units', min_value=16, max_value=128, step=16), activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss=mean_squared_error, optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), metrics=[RootMeanSquaredError()])
    return model

#define hypermodel for GRU tuning
def build_gru_model(hp):
    model = Sequential()
    model.add(InputLayer(input_shape=(window_size, x_train.shape[2])))
    model.add(GRU(units=hp.Int('units', min_value=32, max_value=256, step=32), return_sequences=True))
    model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(GRU(units=hp.Int('units', min_value=32, max_value=256, step=32)))
    model.add(Dense(hp.Int('dense_units', min_value=16, max_value=128, step=16), activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss=mean_squared_error, optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), metrics=[RootMeanSquaredError()])
    return model


In [ ]:
#hyperparameter tuning for each model
objective = kt.Objective("val_root_mean_squared_error", direction="min")

tuner_lstm = kt.Hyperband(build_lstm_model, objective=objective, max_epochs=10, factor=3, directory='my_dir', project_name='lstm_hyperband')
tuner_cnn_lstm = kt.Hyperband(build_cnn_lstm_model, objective=objective, max_epochs=10, factor=3, directory='my_dir', project_name='cnn_lstm_hyperband')
tuner_gru = kt.Hyperband(build_gru_model, objective=objective, max_epochs=10, factor=3, directory='my_dir', project_name='gru_hyperband')

#search for best hyperparameters
tuner_lstm.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val))
tuner_cnn_lstm.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val))
tuner_gru.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val))
#get the best hyperparameters
best_hps_lstm = tuner_lstm.get_best_hyperparameters(num_trials=1)[0]
best_hps_cnn_lstm = tuner_cnn_lstm.get_best_hyperparameters(num_trials=1)[0]
best_hps_gru = tuner_gru.get_best_hyperparameters(num_trials=1)[0]

print(f"LSTM - The optimal number of units in the LSTM layer is {best_hps_lstm.get('units')}, with a dropout rate of {best_hps_lstm.get('dropout')}, and a learning rate of {best_hps_lstm.get('learning_rate')}.")
print(f"CNN-LSTM - The optimal number of filters is {best_hps_cnn_lstm.get('filters')}, kernel size is {best_hps_cnn_lstm.get('kernel_size')}, pooling size is {best_hps_cnn_lstm.get('pool_size')}, units in LSTM layer is {best_hps_cnn_lstm.get('units')}, with a dropout rate of {best_hps_cnn_lstm.get('dropout')}, and a learning rate of {best_hps_cnn_lstm.get('learning_rate')}.")
print(f"GRU - The optimal number of units in the GRU layer is {best_hps_gru.get('units')}, with a dropout rate of {best_hps_gru.get('dropout')}, and a learning rate of {best_hps_gru.get('learning_rate')}.")



Reloading Tuner from my_dir/lstm_hyperband/tuner0.json
Reloading Tuner from my_dir/cnn_lstm_hyperband/tuner0.json
Reloading Tuner from my_dir/gru_hyperband/tuner0.json
LSTM - The optimal number of units in the LSTM layer is 192, with a dropout rate of 0.1, and a learning rate of 0.001.
CNN-LSTM - The optimal number of filters is 96, kernel size is 4, pooling size is 4, units in LSTM layer is 192, with a dropout rate of 0.2, and a learning rate of 0.001.
GRU - The optimal number of units in the GRU layer is 64, with a dropout rate of 0.2, and a learning rate of 0.001.


In [ ]:
#building models
model_lstm = tuner_lstm.hypermodel.build(best_hps_lstm)
model_cnn_lstm = tuner_cnn_lstm.hypermodel.build(best_hps_cnn_lstm)
model_gru = tuner_gru.hypermodel.build(best_hps_gru)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
# model training
history_lstm = model_lstm.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=32, callbacks=[ModelCheckpoint('best_model_lstm.keras', save_best_only=True)])
history_cnn_lstm = model_cnn_lstm.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=32, callbacks=[ModelCheckpoint('best_model_cnn_lstm.keras', save_best_only=True)])
history_gru = model_gru.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=32, callbacks=[ModelCheckpoint('best_model_gru.keras', save_best_only=True)])



Epoch 1/50
1051/1051 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 509255.5625 - root_mean_squared_error: 675.2698 - val_loss: 41148.9180 - val_root_mean_squared_error: 202.8520
Epoch 2/50
1051/1051 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 36971.4180 - root_mean_squared_error: 192.2782 - val_loss: 15388.2861 - val_root_mean_squared_error: 124.0495
Epoch 3/50
1051/1051 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 3607.5039 - root_mean_squared_error: 57.6081 - val_loss: 1372.5297 - val_root_mean_squared_error: 37.0477
Epoch 4/50
1051/1051 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 716.2493 - root_mean_squared_error: 26.7596 - val_loss: 985.9614 - val_root_mean_squared_error: 31.4000
Epoch 5/50
1051/1051 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 748.0789 - root_mean_squared_error: 27.2013 - val_loss: 728.2017 - val_root_mean_squared_error: 26.9852
Epoch 6/50
1051/1051 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 526.2811 - root_mean_squared_error: 22.9254 - val_loss: 617.6216 - val_root_mean_squared_

ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=best_model_cnn_lstm.h5

In [ ]:
#evaluation of the models
def evaluate_model(model, x_test, y_test):
    predictions = model.predict(x_test).flatten()
    mse_value = mean_squared_error(y_test, predictions)
    rmse_value = np.sqrt(mse_value)
    return mse_value, rmse_value

mse_lstm, rmse_lstm = evaluate_model(model_lstm, x_test, y_test)
mse_cnn_lstm, rmse_cnn_lstm = evaluate_model(model_cnn_lstm, x_test, y_test)
mse_gru, rmse_gru = evaluate_model(model_gru, x_test, y_test)

print(f"LSTM Model - MSE: {mse_lstm}, RMSE: {rmse_lstm}")
print(f"CNN-LSTM Model - MSE: {mse_cnn_lstm}, RMSE: {rmse_cnn_lstm}")
print(f"GRU Model - MSE: {mse_gru}, RMSE: {rmse_gru}")

In [ ]:
#function to plot predictions
def plot_predictions(model, X, y, start=0, end=100):
    predictions = model.predict(X).flatten()
    df = pd.DataFrame(data={'Predictions': predictions, 'Actuals': y})
    plt.plot(df['Predictions'][start:end], label='Predictions')
    plt.plot(df['Actuals'][start:end], label='Actuals')
    plt.legend()
    plt.show()


In [ ]:
#plot predictions for each model
print(f"Plotting predictions for LSTM Model")
plot_predictions(model_lstm, x_test, y_test)
print(f"Plotting predictions for CNN-LSTM Model")
plot_predictions(model_cnn_lstm, x_test, y_test)
print(f"Plotting predictions for GRU Model")
plot_predictions(model_gru, x_test, y_test)